In [44]:
from langchain_community.document_loaders import TextLoader # transform book description into langchain document
from langchain_text_splitters import RecursiveCharacterTextSplitter # split long description into smaller chunks
# from langchain_openai import OpenAIEmbeddings # embedding chunks
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma # vector database

In [45]:
from dotenv import load_dotenv
load_dotenv()

True

In [46]:
import pandas as pd
books = pd.read_csv('cleaned_books.csv')

In [47]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [48]:
books["tagged_description"].to_csv("books_description.txt",
                                   sep="\n",
                                   index=False,
                                   header=False)

In [49]:
raw_document = TextLoader("books_description.txt").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0,separators=["\n"])
documents = text_splitter.split_documents(raw_document)

In [50]:
documents[0]

Document(metadata={'source': 'books_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gil

In [51]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Administrator\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Administrator\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [52]:
db_books = Chroma.from_documents(documents, embedding=hf_embeddings)

In [53]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k=10)
docs

[Document(id='70022476-ae38-4a77-9dcc-cde685a08fc9', metadata={'source': 'books_description.txt'}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects.\n9780786808397 Introduce your baby to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to exopse little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing baby to some basic -- and sometimes playful -- information about the subjects.\n9780786808717 A very special puddle sets Violet the mouse off o

In [54]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3749,9780786808380,0786808381,Baby Einstein: Babies,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=jv4NA...,"Introduce your babies to birds, cats, dogs, an...",2002.0,4.03,20.0,29.0,Baby Einstein: Babies,"9780786808380 Introduce your babies to birds, ..."


In [55]:
def retrieve_semantic_recommandation(query, topk=10) -> pd.DataFrame:
    # Retrieve documents from the vector store
    docs = db_books.similarity_search(query, k=50)
    
    results = []
    for doc in docs:
        try:
            # Split by whitespace and take the first element (the ISBN)
            isbn_str = doc.page_content.split()[0]
            results.append(int(isbn_str))
        except (ValueError, IndexError):
            # Safely skip this document if it's empty or doesn't start with a number
            continue
    # print("top 1 description:", docs[0].page_content)
    # Filter the original dataframe and return the top results
    return books[books["isbn13"].isin(results)].head(topk)

In [56]:
retrieve_semantic_recommandation("A book about love and relationship")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
71,9780007189953,0007189958,Where Rainbows End,Cecelia Ahern,Friendship,http://books.google.com/books/content?id=PA7t6...,The new warm and absorbing story from the auth...,2004.0,3.94,454.0,642.0,Where Rainbows End,9780007189953 The new warm and absorbing story...
155,9780060572976,0060572973,"The Good, the Bad, and the Undead",Kim Harrison,Fiction,http://books.google.com/books/content?id=-hBqy...,"It's a tough life for witch Rachel Morgan, sex...",2005.0,4.26,453.0,53112.0,"The Good, the Bad, and the Undead",9780060572976 It's a tough life for witch Rach...
191,9780060654856,0060654856,Learning to Love,Thomas Merton,Religion,http://books.google.com/books/content?id=U0nXs...,Having embraced a life of solitude in his own ...,1998.0,4.50,382.0,15.0,Learning to Love: Exploring Solitude and Freedom,9780060654856 Having embraced a life of solitu...
214,9780060760250,0060760257,Charms for the Easy Life,Kaye Gibbons,Fiction,http://books.google.com/books/content?id=TE2mB...,"A family without men, the Birches live gloriou...",2005.0,4.02,272.0,10983.0,Charms for the Easy Life,"9780060760250 A family without men, the Birche..."
215,9780060760441,0060760443,The Reading Group,Elizabeth Noble,Fiction,http://books.google.com/books/content?id=IagWj...,The Reading Group follows the trials and tribu...,2005.0,3.34,429.0,6408.0,The Reading Group: A Novel,9780060760441 The Reading Group follows the tr...
285,9780060916466,006091646X,The Dance of Intimacy,Harriet Lerner,Psychology,http://books.google.com/books/content?id=tTKc8...,"In The Dance of Intimacy, the bestselling auth...",1990.0,4.06,255.0,7128.0,The Dance of Intimacy: A Woman's Guide to Cour...,"9780060916466 In The Dance of Intimacy, the be..."
292,9780060925758,0060925752,Soul Mates,Thomas Moore,Psychology,http://books.google.com/books/content?id=7syEl...,This companion volume to Care of the Soul offe...,1994.0,4.00,288.0,4122.0,Soul Mates,9780060925758 This companion volume to Care of...
362,9780061120060,0061120065,Their Eyes Were Watching God,Zora Neale Hurston,Fiction,http://books.google.com/books/content?id=Gz-d8...,One of the most important works of twentieth-c...,2006.0,3.90,219.0,209110.0,Their Eyes Were Watching God,9780061120060 One of the most important works ...
365,9780061122095,0061122092,By the River Piedra I Sat Down and Wept,Paulo Coelho,Fiction,http://books.google.com/books/content?id=9AHal...,"From Paulo Coelho, author of the international...",2006.0,3.57,208.0,68403.0,By the River Piedra I Sat Down and Wept: A Nov...,"9780061122095 From Paulo Coelho, author of the..."
